In [83]:
import os
import re
import numpy

In [96]:
PATHS = {
    'TRAIN' : {
        'POSITIVE' : './IMDB_dataset/train/pos',
        'NEGATIVE' : './IMDB_dataset/train/neg'
    },
    'TEST' : {
        'POSITIVE' : './IMDB_dataset/test/pos',
        'NEGATIVE' : './IMDB_dataset/test/neg'
    }
}

documents_amount = {
    'TRAIN' : {
        'POSITIVE' : len(os.listdir(PATHS['TRAIN']['POSITIVE'])),
        'NEGATIVE' : len(os.listdir(PATHS['TRAIN']['NEGATIVE']))
    }, 
    'TEST' : {
        'POSITIVE' : len(os.listdir(PATHS['TEST']['POSITIVE'])),
        'NEGATIVE' : len(os.listdir(PATHS['TEST']['NEGATIVE']))
    }
}

total_documents = {
    'TRAIN' : sum(documents_amount['TRAIN'].values()),
    'TEST'  : sum(documents_amount['TEST'].values())
}

classes = {
    'POSITIVE' : {
        'WORDS' : [],
        'FREQUENCY' : {},
        'PRIOR': 1
    },
    'NEGATIVE' : {
        'WORDS' : [],
        'FREQUENCY' : {},
        'PRIOR': 1
    }
}

for tipo, total in total_documents.items():
    print(f"Total de textos ({tipo}):\n{total}")

for classe in classes.keys():
    print(f"\nTotal de textos com avaliação {classe}\n{documents_amount['TRAIN'][classe]}")
    for fileName in os.listdir(PATHS['TRAIN'][classe])[0:5]:
        print(fileName)

Total de textos (TRAIN):
25000
Total de textos (TEST):
25000

Total de textos com avaliação POSITIVE
12500
3154_10.txt
1760_10.txt
8067_8.txt
6038_10.txt
6447_8.txt

Total de textos com avaliação NEGATIVE
12500
8466_1.txt
7084_1.txt
1102_3.txt
1389_2.txt
2857_1.txt


In [97]:
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+"
CORPUS_USE_PERCENTAGE = 0.1 # usando 100%, treino demora +-35 min

def get_document_words(document_path):
    content = open(document_path, 'r').read().lower()
    return set(re.findall(regex, content))

def update_document(dir, words_list):
    for i in range(0, int(len(os.listdir(dir))*CORPUS_USE_PERCENTAGE)):
        file_name = os.listdir(dir)[i]
        words    = get_document_words(dir+"/"+file_name)
        words_list.extend(words)

# for classe in classes.keys():
#     update_document(PATHS['TRAIN'][classe], classes[classe]['WORDS'])
#     print(f"Quantidade de palavras nos documentos {classe}:\n{len(classes[classe]['WORDS'])}") 

# print("Resultado da extração:\n{}".format(documents["3154_10.txt"]))   

In [98]:
def updateFrequency(frequency, words):
    for word in set(words):
        frequency[word] = words.count(word)

# for classe in classes.keys():
#     updateFrequency(classes[classe]['FREQUENCY'], classes[classe]['WORDS'])
#     print(f"Frequência da palavra FUNNY na classe {classe}:\n{classes[classe]['FREQUENCY']['funny']}")

# vocabulay = set().union(*documents.values())
# print(len(vocabulay))

In [102]:
LAPLACIAN_SMOOTHING = 1

def likelihood_word_class(word, classe):
    count_word_class = 0
    if (word in classes[classe]['FREQUENCY']):
        count_word_class = classes[classe]['FREQUENCY'][word]
    
    total_words_class = len(classes[classe]['WORDS'])
    
    return (count_word_class + LAPLACIAN_SMOOTHING)/((LAPLACIAN_SMOOTHING + 1) * total_words_class)

def likelihood_doc_class(doc_path):
    likelihood = {}
                            
    for classe in classes.keys():
        likelihood[classe] = 1
                            
        for word in get_document_words(doc_path):
            likelihood[classe] *= likelihood_word_class(word, classe)
    
    return max(likelihood, key=likelihood.get)

In [99]:
def train():    
    for classe in classes.keys():
        update_document(PATHS['TRAIN'][classe], classes[classe]['WORDS'])
        updateFrequency(classes[classe]['FREQUENCY'], classes[classe]['WORDS'])
        classes[classe]['PRIOR'] = documents_amount['TRAIN'][classe]/total_documents['TRAIN']

train()

In [103]:
def test():
    correct_tests = 0
                            
    for classe in classes.keys():
        path = PATHS['TEST'][classe]
        
        for i in range(0, int(len(os.listdir(path))*CORPUS_USE_PERCENTAGE)):
            file_name = os.listdir(path)[i]
            if (classe == likelihood_doc_class(path+"/"+file_name)):
                correct_tests += 1
                            
    print(f"Accuracy = {100*correct_tests/total_documents['TEST']}%")
        
test()

Accuracy = 5.804%
